In [ ]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(4)
# mesh = Mesh(unit_cube.GenerateMesh(maxh=0.3))

In [ ]:
fesl2 = SurfaceL2(mesh, order=0, dual_mapping=True)
ul2,vl2 = fesl2.TnT()

fesh1 = H1(mesh, order=1)
uh1, vh1 = fesh1.TnT()

a = BilinearForm(uh1*vl2*ds).Assemble()
print (a.mat)